# EMA - Stable Isotope Labeling Analysis Notebook

### Parameters
The next code block sets parameters that are used throughout the remainder of the notebook.

In [ ]:
# set metatlas generated output parameters 
project_directory = "/global/homes/t/tharwood/metabolomics_data/"
experiment = "20240111_JGI_EY_508352_Set1Set2_final1_EXP120B_HILICZ_USHXG02558_Set1"
polarity = "positive" # 'positive' or 'negative'
workflow_name = "JGI-HILIC"
rt_alignment_number = 2
analysis_number = 2

short_group_pairs = [('POS_D-Light-No-DCMU', 'POS_F-Light-13Cbicarb-DCMU'),
                     ('POS_C-Light-No-Ctrl-EtOH', 'POS_E-Light-13Cbicarb-Ctrl-EtOH'),
                     ('POS_A-Light-No-No', 'POS_B-Light-13Cbicarb-No')]

In [ ]:
# pylint: disable=wrong-import-position,import-error,missing-class-docstring
parameters = {k: v for k, v in globals().items() if k[0] != "_" and k not in ["In", "Out", "get_ipython", "exit", "quit", "open"]}

import logging  # noqa: E402
from pathlib import Path  # noqa: E402


class StopExecution(Exception):
    def _render_traceback_(self):
        pass


logger = logging.getLogger("metatlas.jupyter")
kernel_def = """{"argv":["shifter","--entrypoint","--image=ghcr.io/biorack/metatlas/metatlas_shifter:latest","/usr/local/bin/python","-m",
                 "ipykernel_launcher","-f","{connection_file}"],"display_name": "Metatlas Targeted","language": "python",
                 "metadata": { "debugger": true }}"""
kernel_file_name = Path.home() / ".local" / "share" / "jupyter" / "kernels" / "metatlas-targeted" / "kernel.json"
try:
    has_root_kernel = Path("/root/.local/share/jupyter/kernels/papermill/kernel.json").is_file()
except PermissionError:
    has_root_kernel = False
if not has_root_kernel and not kernel_file_name.is_file():
    kernel_file_name.parent.mkdir(parents=True, exist_ok=True)
    with kernel_file_name.open(mode="w", encoding="utf-8") as f:
        f.writelines(kernel_def)
    logger.critical('CRITICAL: Notebook kernel has been installed. Set kernel to "Metatlas Targeted" and re-run notebook.')
    raise StopExecution
try:
    from analysis_tools.stable_isotope_analysis import generate_outputs  # noqa: E402
    from analysis_tools.noise_detection_gui import Dashboard  # noqa: E402
except ImportError as err:
    logger.critical('CRITICAL: Set notebook kernel to "Metatlas Targeted" and re-run notebook.')
    raise StopExecution from err

In [ ]:
compound_data, output_path = generate_outputs(
                                     project_directory,
                                     experiment,
                                     polarity,
                                     workflow_name,
                                     rt_alignment_number,
                                     analysis_number,
                                     short_group_pairs
)

In [ ]:
dash = Dashboard(
    compound_data,
    output_path,
    compound_idx=0,
    plot_size=(1200, 800)
)